##Content
build up classification models for the cat and dog owners from text comments
* Upload and Import Data

## Upload and Import Data

In [0]:
file_location = "/FileStore/tables/animals_comments_csv.gz"
df = spark.read.load(file_location, format='csv', header = True, inferSchema = True)
df.show(10)

+--------------------+------+-------------------------------------+
|        creator_name|userid|                              comment|
+--------------------+------+-------------------------------------+
|        Doug The Pug|  87.0|                 I shared this to ...|
|        Doug The Pug|  87.0|                   Super cute  😀🐕🐶|
|         bulletproof| 530.0|                 stop saying get e...|
|       Meu Zoológico| 670.0|                 Tenho uma jiboia ...|
|              ojatro|1031.0|                 I wanna see what ...|
|     Tingle Triggers|1212.0|                 Well shit now Im ...|
|Hope For Paws - O...|1806.0|                 when I saw the en...|
|Hope For Paws - O...|2036.0|                 Holy crap. That i...|
|          Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
|       Brian Barczyk|2698.0|                 Call the teddy Larry|
+--------------------+------+-------------------------------------+
only showing top 10 rows



## Data Preprocessing

In [0]:
df.dtypes

Out[2]: [('creator_name', 'string'), ('userid', 'double'), ('comment', 'string')]

In [0]:
print("Number of rows in df:", df.count())

Number of rows in df: 5820035


In [0]:
# Count null values in each columns 
# The null values are less than 1% of the total data, they are removed
print('Number of null values in creator_name: ',df.filter(df['creator_name'].isNull()).count())
print('Number of null values in userid: ',df.filter(df['userid'].isNull()).count())
print('Number of null values in comment: ',df.filter(df['comment'].isNull()).count())

Number of null values in creator_name:  32050
Number of null values in userid:  565
Number of null values in comment:  1051


### Removing Missing Data

In [0]:
#remove rows with no comments and no userid
def remove_null(df):
    df_drop = df.filter(df['comment'].isNotNull())
    df_drop = df_drop.filter(df_drop['userid'].isNotNull())
    df_drop = df_drop.dropDuplicates()
  
    print('After dropping, we have ', str(df_drop.count()), 'row in dataframe')
    return df_drop
df = remove_null(df)

After dropping, we have  5757212 row in dataframe


### Covert all comments to lower case

In [0]:
import pyspark.sql.functions as F
df = df.withColumn('comment', F.lower(F.col('comment')))
display(df)

creator_name,userid,comment
Brian Barczyk,28322.0,since i watched these snake videos i am not afraid of snakes anymore and now i really want a snake for my birthday
WaysideWaifs,89939.0,why are humans so inhuman...
MaxluvsMya,3267.0,why are you always awake so late lol
Brave Wilderness,130178.0,what is worst noni or the bullet ant sting?
Hope For Paws - Official Rescue Channel,1806.0,when i saw the end it said to adopt i saw different animal sites i was mad that they separated the cute little pups after being together for a long time
Brian Barczyk,2698.0,call the teddy larry
The Dodo,24401.0,u need a scuba tank so you can stay down longer
BarkBox,41679.0,well never stop getting barkbox for our dog he likes it too much. does anyone know if there is a barkbox option to get just one toy a month? thats all our dog can handle
RaleighLink14,65992.0,since bratayley put up this video annie only went to see her horse 2 times. they travel to much and i totally agree what raleigh said. also green horse and green rider... it isnt a great mix. im not being rude its just my opinion. xx
Viktor Larkhill,89939.0,omg.. i pray the doggies get well soon and finds lovely forever homes.... viktor you are an inspiration to many... god bless you..


### Data Sampling (around 10% data)

In [0]:
# To preprocessing all data and train models cost too long for free databrick version. only use 500000 rows of data for this project
from pyspark.sql.functions import rand 
df.orderBy(rand(seed=0)).createOrReplaceTempView("data")
df_sample = spark.sql("select * from data limit 500000")
display(df_sample)

creator_name,userid,comment
Waggle TV,935462.0,omg so cuuuttteeee! i my heart melted 😍 😍 😍 😍 😍 🐱🐱🐱🐱🐈🐈🐈🐈
Brave Wilderness,823749.0,will i be with you next time?
The Pet Collective,178914.0,is it stopped? lol
Talking Kitty Cat,2370875.0,do another vid
Brave Wilderness,1018655.0,you should try ofc if your in ohio
Info Marvel,132417.0,quiero un funko xd
Hope For Paws - Official Rescue Channel,2267432.0,aww this video is so sweet...i was crying when i saw the name😢
Brave Wilderness,854684.0,and it was big as that one
Ericas Slot World,32528.0,awesome wins
Shehry Vlogs,1269044.0,happy ramzan😃🌹💮ramzan mubarak.....


In [0]:
df_sample.count()

Out[8]: 500000

###Construct Label Feature

In [0]:
# find user with preference of dog and cat
cond = (df_sample["comment"].like("%my dog%") | df_sample["comment"].like("%i have a dog%")\
        | df_sample["comment"].like("%my cat%") | df_sample["comment"].like("%i have a cat%") \
        | df_sample["comment"].like("%my dogs%") | df_sample["comment"].like("%my cats%")\
        | df_sample["comment"].like("%my cat%") | df_sample["comment"].like("%i have dogs%")\
        | df_sample["comment"].like("%i have cats%") | df_sample["comment"].like("%my puppy%")\
        | df_sample["comment"].like("%my kitten%") | df_sample["comment"].like("%i have a puppy%")\
        | df_sample["comment"].like("%i have puppies%"))

df_sample = df_sample.withColumn('dog_cat',  cond)
df_sample = df_sample.withColumn('no_pet', ~df_sample["comment"].like("%my%") & ~df_sample["comment"].like("%have%") & ~df_sample["comment"].like("%my dog%") \
                              & ~df_sample["comment"].like("%my cat%")) 

display(df_sample)

creator_name,userid,comment,dog_cat,no_pet
Waggle TV,935462.0,omg so cuuuttteeee! i my heart melted 😍 😍 😍 😍 😍 🐱🐱🐱🐱🐈🐈🐈🐈,false,false
Brave Wilderness,823749.0,will i be with you next time?,false,true
The Pet Collective,178914.0,is it stopped? lol,false,true
Talking Kitty Cat,2370875.0,do another vid,false,true
Brave Wilderness,1018655.0,you should try ofc if your in ohio,false,true
Info Marvel,132417.0,quiero un funko xd,false,true
Hope For Paws - Official Rescue Channel,2267432.0,aww this video is so sweet...i was crying when i saw the name😢,false,true
Brave Wilderness,854684.0,and it was big as that one,false,true
Ericas Slot World,32528.0,awesome wins,false,true
Shehry Vlogs,1269044.0,happy ramzan😃🌹💮ramzan mubarak.....,false,true


### Tokenizing

In [0]:
from pyspark.ml.feature import RegexTokenizer, Word2Vec
regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="words", pattern="\\W")
df_sample = regexTokenizer.transform(df_sample)
df_sample.show(10)

+--------------------+---------+--------------------+-------+------+--------------------+
|        creator_name|   userid|             comment|dog_cat|no_pet|               words|
+--------------------+---------+--------------------+-------+------+--------------------+
|           Waggle TV| 935462.0|omg so cuuuttteee...|  false| false|[omg, so, cuuuttt...|
|    Brave Wilderness| 823749.0|will i be with yo...|  false|  true|[will, i, be, wit...|
|  The Pet Collective| 178914.0|  is it stopped? lol|  false|  true|[is, it, stopped,...|
|   Talking Kitty Cat|2370875.0|      do another vid|  false|  true|  [do, another, vid]|
|    Brave Wilderness|1018655.0|you should try  o...|  false|  true|[you, should, try...|
|         Info Marvel| 132417.0|  quiero un funko xd|  false|  true|[quiero, un, funk...|
|Hope For Paws - O...|2267432.0|aww this video is...|  false|  true|[aww, this, video...|
|    Brave Wilderness| 854684.0|and it was big as...|  false|  true|[and, it, was, bi...|
|   Ericas

### Obatain Word Vector using Word2vect

In [0]:
word2Vec = Word2Vec(vectorSize=30, minCount=1, inputCol="words", outputCol="wordVector")
model = word2Vec.fit(df_sample)
data= model.transform(df_sample)
display(data)

creator_name,userid,comment,dog_cat,no_pet,words,wordVector
Waggle TV,935462.0,omg so cuuuttteeee! i my heart melted 😍 😍 😍 😍 😍 🐱🐱🐱🐱🐈🐈🐈🐈,false,false,"List(omg, so, cuuuttteeee, i, my, heart, melted)","Map(vectorType -> dense, length -> 30, values -> List(-0.07466187940112182, -0.20838288410699793, 0.1324264944830377, -0.024825547555727617, -0.12878190326903546, 0.03794203592198236, 0.27583101764321327, 0.04214263265021145, -0.10951728985777923, -0.0470111785190446, -0.012633550113865306, 0.187299339965518, -0.13388548059655087, 0.08390451595187187, -0.050774987787008286, -0.14597763047952736, -0.08779163957972612, 0.03465812662450064, -0.17476791170026573, -0.0288113286452634, -0.1104191640791084, -0.21200774902743952, 0.09595432797712938, 0.21904150902160574, 0.2589101195335388, -0.07689503580331802, -0.2554157045004623, -0.4088724540280444, -0.0403162034095398, 0.15587798372975417))"
Brave Wilderness,823749.0,will i be with you next time?,false,true,"List(will, i, be, with, you, next, time)","Map(vectorType -> dense, length -> 30, values -> List(-0.15598009253985115, -0.14778589191181318, 0.015028277678149087, -0.0442960544356278, 0.07165342409695898, -0.09080295317939349, 0.0918543870959963, -0.015054452632154736, -0.25161305389233996, -0.09471620725733892, -0.07438113221100398, 0.26989313015448196, 0.2526968690965857, -0.4030399693708334, 0.07358090552900519, -0.10044817067682743, 0.07172632696373121, 0.204570255109242, -0.21608497548316205, 0.09086645261517592, 0.15106882687125886, 0.0899548019681658, 0.08376904683453695, -0.06153537213270153, -0.19207279809883662, -0.15848060431224958, -0.1900717381920133, -0.03134851264102118, 0.012265282550028392, -0.05460901611617633))"
The Pet Collective,178914.0,is it stopped? lol,false,true,"List(is, it, stopped, lol)","Map(vectorType -> dense, length -> 30, values -> List(-0.016521641286090016, -0.18212509714066982, 0.17548917466774583, -0.07953933178214356, -0.1193255740654422, -0.13871250208467245, 0.27975744009017944, 0.18910090997815132, -0.08840985130518675, -0.2557426504790783, 0.1024506411049515, 0.26153228245675564, 0.27808234794065356, 0.029358407482504845, 0.05377034191042185, 0.012973509263247252, -0.0032087923027575016, -0.09663892351090908, -0.3679237812757492, 0.06315898150205612, 0.17904045432806015, -0.006913928315043449, -0.020719077438116074, 0.002955511212348938, 0.09306730143725872, 0.010158304125070572, -0.026430388912558556, -0.09001512243412435, -0.12717261794023216, 0.06006549345329404))"
Talking Kitty Cat,2370875.0,do another vid,false,true,"List(do, another, vid)","Map(vectorType -> dense, length -> 30, values -> List(-0.15587492287158966, -0.13115408147374788, 0.042994365096092224, -0.07564760247866312, -0.1827316315223773, -0.015837964912255604, -0.05348749707142512, -0.23304010927677155, -0.1670026332139969, 0.07268123825391133, 0.1043395921587944, -0.017412965496381123, 0.20958862950404483, -0.21023389200369516, -0.12187538792689641, 0.03374408682187398, 0.21416370570659637, 0.12094537013520797, -0.24724893644452095, 0.3040754795074463, 0.12318279842535654, 0.5078013042608897, 0.04353859151403109, -0.040791230276227, 0.2529054780801137, -0.050668080647786454, -0.2665165600677331, 0.0848734900355339, -0.1322258673608303, -0.3244616960485776))"
Brave Wilderness,1018655.0,you should try ofc if your in ohio,false,true,"List(you, should, try, ofc, if, your, in, ohio)","Map(vectorType -> dense, length -> 30, values -> List(-0.1768311089836061, -0.10910743288695812, -0.047438159002922475, -0.09360082124476321, -0.13761433097533882, -0.03112967498600483, -0.006305700575467199, 0.040213123662397265, -0.11776560975704342, 0.06069771503098309, -0.24581995548214763, 0.42824289202690125, 0.08176304576772964, -0.25065838173031807, 0.11232816893607378, -0.009214894904289395, 0.07649663253687322, 0.11297939528594725, -0.21079476663726382, 0.1600798841973301, -0.040204108954640105, 0.2089224299415946, 0.005775229772552848, -0.051605339627712965, -0.08753122016787529, -

In [0]:
data.count()

Out[12]: 500000

### Fix Imbalance Data  - Under Sample

In [0]:
# negaive lable is 10 times more than positive lable
data_pets = data.filter(F.col('dog_cat') == True) 
data_no_pets = data.filter(F.col('no_pet') ==  True)
print("Number of confirmed user who own dogs or cats: ", data_pets.count())
print("Number of confirmed user who don't have pet's: ", data_no_pets.count())

Number of confirmed user who own dogs or cats:  4572
Number of confirmed user who don't have pet's:  416025


In [0]:
# label ratio = 1/2 
data_no_pets.orderBy(rand()).createOrReplaceTempView("table")

Num_Pos_Label = data.filter(F.col('dog_cat') == True).count() 
Num_Neg_Label = data.filter(F.col('no_pet') ==  True).count()

#Q1 = spark.sql("SELECT col1 from table where col2>500 limit {}, 1".format(q25))
#pass variable to sql
data_no_pets_down = spark.sql("select * from table where limit {}".format(Num_Pos_Label*2))

In [0]:
print('Now after balancing the lables, we have ')   
print('Positive label: ', Num_Pos_Label)
print('Negtive label: ', data_no_pets_down.count())

Now after balancing the lables, we have 
Positive label:  4572
Negtive label:  9144


### Target Feature Encoding

In [0]:
def get_label(data_pets,data_no_pets_down):
    data_labeled = data_pets.select('dog_cat','wordVector').union(data_no_pets_down.select('dog_cat','wordVector'))
    return data_labeled

data_labeled = get_label(data_pets,data_no_pets_down)
data_labeled.show(10)

+-------+--------------------+
|dog_cat|          wordVector|
+-------+--------------------+
|   true|[-0.1902915891259...|
|   true|[-0.0985827284372...|
|   true|[-0.1430899834781...|
|   true|[-0.0264325229864...|
|   true|[-0.3016146466963...|
|   true|[-0.1632387682440...|
|   true|[-0.1459623438882...|
|   true|[-0.1018698478943...|
|   true|[-0.1086992367337...|
|   true|[-0.1035014218746...|
+-------+--------------------+
only showing top 10 rows



In [0]:
data_labeled

Out[20]: DataFrame[dog_cat: boolean, wordVector: vector]

In [0]:
#convert Boolean value to 1 and 0's
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

def multiple(x):
    return int(x*1)
udf_boolToInt= udf(lambda z: multiple(z),IntegerType())
df_labeled = data_labeled.withColumn('label',udf_boolToInt('dog_cat'))
df_labeled.show(10)

+-------+--------------------+-----+
|dog_cat|          wordVector|label|
+-------+--------------------+-----+
|   true|[-0.1902915891259...|    1|
|   true|[-0.0985827284372...|    1|
|   true|[-0.1430899834781...|    1|
|   true|[-0.0264325229864...|    1|
|   true|[-0.3016146466963...|    1|
|   true|[-0.1632387682440...|    1|
|   true|[-0.1459623438882...|    1|
|   true|[-0.1018698478943...|    1|
|   true|[-0.1086992367337...|    1|
|   true|[-0.1035014218746...|    1|
+-------+--------------------+-----+
only showing top 10 rows



## Model Selection and Evaluation

### Training and Test Data Split

In [0]:
train, test = df_labeled.randomSplit([0.8, 0.2], seed=12345)

### Logistic Regression

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier


In [0]:
lr = LogisticRegression(featuresCol="wordVector",labelCol="label" , maxIter=10, regParam=0.1, elasticNetParam=0.8)

# Run TrainValidationSplit, and choose the best set of parameters.
lrModel = lr.fit(train)

# Make predictions on test data. model is the model with combination of parameters
# that performed best.
lr_predictions = lrModel.transform(test)
lr_predictions.show(10)

+-------+--------------------+-----+--------------------+--------------------+----------+
|dog_cat|          wordVector|label|       rawPrediction|         probability|prediction|
+-------+--------------------+-----+--------------------+--------------------+----------+
|   true|[-0.4852680265903...|    1|[0.21563014565240...|[0.55369962770904...|       0.0|
|   true|[-0.4090231597423...|    1|[-0.1617485063958...|[0.45965080498417...|       1.0|
|   true|[-0.4039231002330...|    1|[-0.1790656375623...|[0.45535282602227...|       1.0|
|   true|[-0.3966523755341...|    1|[-0.0708856887650...|[0.48228599460391...|       1.0|
|   true|[-0.3866898834705...|    1|[0.31097878457853...|[0.57712415334824...|       0.0|
|   true|[-0.3690780947605...|    1|[0.20052949025479...|[0.54996505146449...|       0.0|
|   true|[-0.3656601145863...|    1|[-0.1030458086708...|[0.47426131918633...|       1.0|
|   true|[-0.3514966964721...|    1|[-0.5499973971744...|[0.36586501286484...|       1.0|
|   true|[

In [0]:
# Check the ROC-AUC
def get_evaluation_result(predictions):
    evaluator = BinaryClassificationEvaluator(
      labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
    AUC = evaluator.evaluate(predictions)
    TP = predictions[(predictions["label"] == 1) & (predictions["prediction"] == 1.0)].count()
    FP = predictions[(predictions["label"] == 0) & (predictions["prediction"] == 1.0)].count()
    TN = predictions[(predictions["label"] == 0) & (predictions["prediction"] == 0.0)].count()
    FN = predictions[(predictions["label"] == 1) & (predictions["prediction"] == 0.0)].count()

    accuracy = (TP + TN)*1.0 / (TP + FP + TN + FN)
    precision = TP*1.0 / (TP + FP)
    recall = TP*1.0 / (TP + FN)
    print ("Test Accuracy:", accuracy)
    print ("Test Precision:", precision)
    print ("Test Recall:", recall)
    print ("Test AUC of ROC:", AUC)

print("Prediction result summary for Logistic Regression Model:  ")
get_evaluation_result(lr_predictions)

Prediction result summary for Logistic Regression Model:  
Test Accuracy: 0.693745506829619
Test Precision: 0.8
Test Recall: 0.09586056644880174
Test AUC of ROC: 0.9043422209131635


### Random Forest

In [0]:
rf = RandomForestClassifier(labelCol="label", featuresCol="wordVector", numTrees=15)
rf_model = rf.fit(train)
rf_predictions = rf_model.transform(test)
rf_predictions.show(10)

+-------+--------------------+-----+--------------------+--------------------+----------+
|dog_cat|          wordVector|label|       rawPrediction|         probability|prediction|
+-------+--------------------+-----+--------------------+--------------------+----------+
|   true|[-0.4852680265903...|    1|[8.1689355209498,...|[0.54459570139665...|       0.0|
|   true|[-0.4090231597423...|    1|[2.92927524127241...|[0.19528501608482...|       1.0|
|   true|[-0.4039231002330...|    1|[5.55409926311906...|[0.37027328420793...|       1.0|
|   true|[-0.3966523755341...|    1|[4.36597548586913...|[0.29106503239127...|       1.0|
|   true|[-0.3866898834705...|    1|[7.44143199858576...|[0.49609546657238...|       1.0|
|   true|[-0.3690780947605...|    1|[5.94873407172154...|[0.39658227144810...|       1.0|
|   true|[-0.3656601145863...|    1|[4.46494719494271...|[0.29766314632951...|       1.0|
|   true|[-0.3514966964721...|    1|[2.92927524127241...|[0.19528501608482...|       1.0|
|   true|[

In [0]:
print("Prediction result summary for Random Forest Model:  ")
get_evaluation_result(rf_predictions)

Prediction result summary for Random Forest Model:  
Test Accuracy: 0.8838964773544212
Test Precision: 0.8789808917197452
Test Recall: 0.7516339869281046
Test AUC of ROC: 0.9494603635445604
